In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

In [4]:
def data():
    # Split train & validation set
    BBB_Train_Decile1 = pd.read_pickle("BBB_Train_Decile1.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile1.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile1, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 1')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 4s 1ms/step - loss: 0.5560 - acc: 0.7363 - val_loss: 0.5389 - val_acc: 0.7448
Epoch 36/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5580 - acc: 0.7357 - val_loss: 0.5389 - val_acc: 0.7435
Epoch 37/75
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5582 - acc: 0.7366 - val_loss: 0.5389 - val_acc: 0.7435
Epoch 38/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5585 - acc: 0.7292 - val_loss: 0.5388 - val_acc: 0.7448
Epoch 39/75
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5540 - acc: 0.7347 - val_loss: 0.5388 - val_acc: 0.7448
Epoch 40/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5617 - acc: 0.7347 - val_loss: 0.5388 - val_acc: 0.7448
Epoch 41/75
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5609 - acc: 0.7366 - val_loss: 0.5388 - val_acc: 0.7448
Epoch 42/75
3087/3087 [==============================] - 3s 1ms/ste

3087/3087 [==============================] - 4s 1ms/step - loss: 0.5365 - acc: 0.7402 - val_loss: 0.5238 - val_acc: 0.7461

Epoch 00019: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 20/75
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5436 - acc: 0.7392 - val_loss: 0.5235 - val_acc: 0.7461
Epoch 21/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5406 - acc: 0.7399 - val_loss: 0.5236 - val_acc: 0.7461
Epoch 22/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5382 - acc: 0.7418 - val_loss: 0.5236 - val_acc: 0.7474
Epoch 23/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5410 - acc: 0.7392 - val_loss: 0.5236 - val_acc: 0.7461
Epoch 24/75
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5438 - acc: 0.7376 - val_loss: 0.5234 - val_acc: 0.7461

Epoch 00024: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 25/75
3087/3087 [==============================] 

Epoch 3/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5896 - acc: 0.7114 - val_loss: 0.5541 - val_acc: 0.7293
Epoch 4/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5733 - acc: 0.7256 - val_loss: 0.5451 - val_acc: 0.7396
Epoch 5/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5657 - acc: 0.7289 - val_loss: 0.5373 - val_acc: 0.7461
Epoch 6/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5623 - acc: 0.7289 - val_loss: 0.5364 - val_acc: 0.7435
Epoch 7/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5543 - acc: 0.7328 - val_loss: 0.5293 - val_acc: 0.7396
Epoch 8/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5550 - acc: 0.7337 - val_loss: 0.5309 - val_acc: 0.7422
Epoch 9/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5484 - acc: 0.7412 - val_loss: 0.5330 - val_acc: 0.7370
Epoch 10/100
3087/3087 [==============================]

3087/3087 [==============================] - 4s 1ms/step - loss: 0.5375 - acc: 0.7412 - val_loss: 0.5215 - val_acc: 0.7461
Epoch 63/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5353 - acc: 0.7431 - val_loss: 0.5215 - val_acc: 0.7474
Epoch 64/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5335 - acc: 0.7457 - val_loss: 0.5215 - val_acc: 0.7461
Epoch 65/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5360 - acc: 0.7428 - val_loss: 0.5215 - val_acc: 0.7461
Epoch 66/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5324 - acc: 0.7532 - val_loss: 0.5215 - val_acc: 0.7474
Epoch 67/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5382 - acc: 0.7441 - val_loss: 0.5216 - val_acc: 0.7487
Epoch 68/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5419 - acc: 0.7428 - val_loss: 0.5216 - val_acc: 0.7474
Epoch 69/100
3087/3087 [==============================] - 4s 

...

3087/3087 [==============================] - 4s 1ms/step - loss: 0.5656 - acc: 0.7331 - val_loss: 0.5455 - val_acc: 0.7383
Epoch 26/125
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5646 - acc: 0.7370 - val_loss: 0.5450 - val_acc: 0.7383
Epoch 27/125
3087/3087 [==============================] - 3s 859us/step - loss: 0.5623 - acc: 0.7373 - val_loss: 0.5444 - val_acc: 0.7383
Epoch 28/125
3087/3087 [==============================] - 2s 803us/step - loss: 0.5626 - acc: 0.7376 - val_loss: 0.5441 - val_acc: 0.7396
Epoch 29/125
3087/3087 [==============================] - 3s 823us/step - loss: 0.5669 - acc: 0.7350 - val_loss: 0.5436 - val_acc: 0.7409
Epoch 30/125
3087/3087 [==============================] - 3s 827us/step - loss: 0.5606 - acc: 0.7334 - val_loss: 0.5436 - val_acc: 0.7409
Epoch 31/125
3087/3087 [==============================] - 3s 833us/step - loss: 0.5598 - acc: 0.7379 - val_loss: 0.5429 - val_acc: 0.7409
Epoch 32/125
3087/3087 [===========================

3087/3087 [==============================] - 3s 815us/step - loss: 0.5556 - acc: 0.7392 - val_loss: 0.5413 - val_acc: 0.7422
Epoch 85/125
3087/3087 [==============================] - 3s 826us/step - loss: 0.5617 - acc: 0.7357 - val_loss: 0.5412 - val_acc: 0.7422
Epoch 86/125
3087/3087 [==============================] - 2s 795us/step - loss: 0.5559 - acc: 0.7425 - val_loss: 0.5413 - val_acc: 0.7422
Epoch 87/125
3087/3087 [==============================] - 2s 760us/step - loss: 0.5616 - acc: 0.7363 - val_loss: 0.5412 - val_acc: 0.7422
Epoch 88/125
3087/3087 [==============================] - 2s 764us/step - loss: 0.5542 - acc: 0.7389 - val_loss: 0.5412 - val_acc: 0.7422
Epoch 89/125
3087/3087 [==============================] - 2s 783us/step - loss: 0.5567 - acc: 0.7379 - val_loss: 0.5411 - val_acc: 0.7422
Epoch 90/125
3087/3087 [==============================] - 2s 759us/step - loss: 0.5581 - acc: 0.7363 - val_loss: 0.5412 - val_acc: 0.7422
Epoch 91/125
3087/3087 [=======================

3087/3087 [==============================] - 2s 490us/step - loss: 0.5394 - acc: 0.7412 - val_loss: 0.5244 - val_acc: 0.7409
Epoch 18/100
3087/3087 [==============================] - 2s 490us/step - loss: 0.5390 - acc: 0.7396 - val_loss: 0.5233 - val_acc: 0.7422
Epoch 19/100
3087/3087 [==============================] - 2s 488us/step - loss: 0.5370 - acc: 0.7405 - val_loss: 0.5238 - val_acc: 0.7396
Epoch 20/100
3087/3087 [==============================] - 2s 489us/step - loss: 0.5395 - acc: 0.7477 - val_loss: 0.5245 - val_acc: 0.7396
Epoch 21/100
3087/3087 [==============================] - 2s 488us/step - loss: 0.5379 - acc: 0.7386 - val_loss: 0.5235 - val_acc: 0.7409

Epoch 00021: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 22/100
3087/3087 [==============================] - 2s 487us/step - loss: 0.5396 - acc: 0.7467 - val_loss: 0.5235 - val_acc: 0.7422
Epoch 23/100
3087/3087 [==============================] - 2s 489us/step - loss: 0.5417 - acc: 0.7438 - 

3087/3087 [==============================] - 2s 489us/step - loss: 0.5406 - acc: 0.7379 - val_loss: 0.5237 - val_acc: 0.7396
Epoch 76/100
3087/3087 [==============================] - 2s 490us/step - loss: 0.5370 - acc: 0.7464 - val_loss: 0.5236 - val_acc: 0.7396
Epoch 77/100
3087/3087 [==============================] - 2s 489us/step - loss: 0.5407 - acc: 0.7444 - val_loss: 0.5236 - val_acc: 0.7396
Epoch 78/100
3087/3087 [==============================] - 2s 489us/step - loss: 0.5349 - acc: 0.7467 - val_loss: 0.5236 - val_acc: 0.7396
Epoch 79/100
3087/3087 [==============================] - 2s 491us/step - loss: 0.5376 - acc: 0.7421 - val_loss: 0.5237 - val_acc: 0.7396
Epoch 80/100
3087/3087 [==============================] - 2s 488us/step - loss: 0.5344 - acc: 0.7386 - val_loss: 0.5236 - val_acc: 0.7396
Epoch 81/100
3087/3087 [==============================] - 2s 500us/step - loss: 0.5387 - acc: 0.7464 - val_loss: 0.5238 - val_acc: 0.7396
Epoch 82/100
3087/3087 [=======================

3087/3087 [==============================] - 1s 478us/step - loss: 0.5416 - acc: 0.7438 - val_loss: 0.5237 - val_acc: 0.7448
Epoch 34/50
3087/3087 [==============================] - 1s 475us/step - loss: 0.5453 - acc: 0.7415 - val_loss: 0.5238 - val_acc: 0.7461
Epoch 35/50
3087/3087 [==============================] - 1s 479us/step - loss: 0.5392 - acc: 0.7457 - val_loss: 0.5238 - val_acc: 0.7461
Epoch 36/50
3087/3087 [==============================] - 1s 479us/step - loss: 0.5395 - acc: 0.7444 - val_loss: 0.5238 - val_acc: 0.7487
Epoch 37/50
3087/3087 [==============================] - 1s 477us/step - loss: 0.5423 - acc: 0.7396 - val_loss: 0.5239 - val_acc: 0.7487
Epoch 38/50
3087/3087 [==============================] - 1s 475us/step - loss: 0.5410 - acc: 0.7399 - val_loss: 0.5238 - val_acc: 0.7474
Epoch 39/50
3087/3087 [==============================] - 1s 481us/step - loss: 0.5456 - acc: 0.7373 - val_loss: 0.5239 - val_acc: 0.7487
Epoch 40/50
3087/3087 [==============================

In [ ]:
0.8297